# Parser 8.00 - Floating Point Numbers

The goal of this version of the parser is to have it accept [floating point numbers](https://en.wikipedia.org/wiki/Floating-point_arithmetic) in decimal and (just for fun) hexadecimal formats.

### Digression

Digital computers maintain a simple relationship between bits and integers. Except for any sign bit, each bit represents a power of two. The maximum value that can be represented is limited only by the number of bits used. Computer hardware registers have a maximum size. Since it is easier to manipulate integers that fit into a register than integers which don't, this suggests (but does not dictate) a maximum integer value supported by any given programming language on any given machine.

Within whatever limits are supported, integer arithmetic is exact. The results of operations on any two integers is always another integer.

Floating point numbers are different. They recast how the bits that represent a number are interpreted to include fractional values. Most often there is also an increase in the range of numbers represented, to far beyond the limits of integer arithmetic using the same number of bits. However (and this is a big however) this is coupled with an *inescapable loss of precision*. Arithmetic operations are no longer guaranteed to result in exactly representable values.

So here be dragons, notoriously difficult creatures to study but whose complex ethology has been [documented](https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html) from time to time. Even without ever encountering one, a few moments of reflection should be enough to convince that they exist.

The crux of the problem is this: the mathematical real numbers are continuous, so that between any two a third can be found, and they are infinite in extent, so that for any one a larger or smaller one can be found. The nature of digital computers, which handle data in discrete and finite chunks, prevents them from directly manipulating real numbers. What they *can* do is *approximate* the real numbers to whatever range and precision we are willing to spend the time and space to achieve. But it is important to remember that no digital approximation can represent more than a tiny subset of the real numbers.

Consider the popular [IEEE 754 double precision](https://en.wikipedia.org/wiki/Double-precision_floating-point_format) floating point standard. Numbers are represented by a 64-bit code:

```
sign    
  < exponent><                  mantissa                        >
 |
 SEEEEEEEEEEEMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
 |                                                              |
Bit 63                                                        Bit 0
```

One bit for the mantissa sign, 11 bits for the exponent, and 52 bits for the mantissa itself. This format has a range of about 10\*\*-308 to 10\*\*308 and a precision of about 15 decimal digits.

>A signed integer using the same 64 bits has a range of about 10\*\*-19 to 10\*\*19 and represents precisely every integer in that range to about 19 decimal digits.

The floating point mantissa actually has 53 bits. Values are stored in normalized form, ie., the mantissa is shifted to the left until the most significant bit is a one bit. The first benefit of this is that each value is guaranteed a unique representation. Second, since the most significant bit is known to be the constant one, without loss of precision it can be dropped from the actual representation and replaced by the sign bit.

The exponent is also signed, with a range of -1022 to 1023. However it is stored as an unsigned value in the range one to 2046 by adding a bias of 1023 to its actual value. This is a computational advantage in this application (eg., exponent magnitudes are easy to compare, so in many cases it is easy to tell which of two values is larger without ever examining the mantissa).

This form of representation is quite granular in its precision and has a very non-uniform density over its range. This is not specific to the IEEE 754 standard but is characteristic of any similar representation.

A 53-bit mantissa can exactly represent

```Python
2**53 = 9007199254740992
```

real numbers between any two consecutive exponent values. This is very large but hardly infinite count. It is also (by no coincidence) the same as the count of integers between 2\*\*53 and 2\*\*54. Exponent values greater than 54 cover ranges in which not even every integer can be represented. Put another way, integers in the range 2\*\*54 to 2\*\*1023 require at least 16 and on up to more than 300 decimal digits to express exactly.

>It should be noted that while astronomically large numbers are quite imprecise in this representation, this is also the case for numbers of similar size in any common representation. A number expressed in [scientific notation](https://en.wikipedia.org/wiki/Scientific_notation) rarely has as many as 15 significant digits, for instance.

At the other end of the scale, half the allowed exponent values are negative. Negative exponents always signify a value less than one. Since the mantissa can be either positive or negative, this means fully half of all the real numbers that can be exactly represented fall between minus one and one. This is true for any similar representation as it depends only on the fact that the exponent is signed.

Two floating point numbers are by far easiest to add or subtract if their exponents have the same value. If they don't, normally they are transparently brought into agreement by whatever software or hardware is manipulating them before attempting any arithmetic.

However exponent alignment can also lead to a loss of precision which may go unnoticed by the user – one of the dragons mentioned earlier. As one exponent is increased or reduced to match the other, its associated mantissa must be adjusted to maintain the value of the number represented. A base 10 example:

```Python
1.0 * 10**3 = 10.0 * 10**2 = 100.0 * 10**1 = 1000.0 * 10**0 = 10000.0 * 10**-1 = 100000.0 * 10**-2 = 1000000 * 10**-3
```

In base two the adjustment is quite simple, as it merely requires shifting the mantissa one bit left or right each time the exponent value changes by one. Knowing the mechanics makes it easy to force a precision loss:

In [1]:
def shred():
    '''force precision loss in floating point arithmetic'''
    
    def shr(i, dlt):
        # should result in 2**i
        e = i + mantissabits + dlt
        return 2**e + 2**i - 2**e

    # for IEEE 754 double

    mantissabits = 53

    # table header
    
    print( f'   i       2**i    Diff={mantissabits-1}    Diff={mantissabits}    Diff={mantissabits+1}\n')

    # table data

    for i in range(-5,6):
        print(f'  {i:>2} {2**i:>10} {shr(i, -1):>10} {shr(i,0):>10} {shr(i,1):>10}' )
        
# uncomment 'shred()' and run this cell to reproduce result

# shred()

   i       2**i    Diff=52    Diff=53    Diff=54

  -5    0.03125    0.03125        0.0        0.0
  -4     0.0625     0.0625        0.0        0.0
  -3      0.125      0.125        0.0        0.0
  -2       0.25       0.25        0.0        0.0
  -1        0.5        0.5        0.0        0.0
   0          1          1          1          1
   1          2          2          2          2
   2          4          4          4          4
   3          8          8          8          8
   4         16         16         16         16
   5         32         32         32         32


Mathematically the values in all four columns should be the same across each row - namely 2 to the power *i* - but in practice this is not always the actual result. A precision loss can readily occur when the number of mantissa shifts required to align the exponents meets or exceeds the number of bits it has. What the result then becomes depends on the hardware and software in use. *Something* will always happen, but exactly what that might be can be hard to predict.

The short answer to dealing with this possibility is to never allow floating point operands of widely varying magnitude to interact with each other. The larger point to be made is that a floating point representation should never be pushed to its limits, as meaningless answers are all too likely to result.

## Libraries

In [ ]:
import glob       # for searching directories

import re         # for regular exprssions

## User output

In [ ]:
visSep = '-------------'             # visual separator

def UIwriteln(this):
    '''write a single line to output'''
    print( f'{this}\n' )
    
def UIwriteSep():
    '''write a visual separator'''
    UIwriteln( visSep )

def UIshow(tag, value):
    '''write a tagged value to output'''
    UIwriteln( f'{tag}: {value}' )

def UIerror(this):
    '''write an error message to output'''
    UIshow( 'Error', this )

# Tracing

In [ ]:
# flags: show trace of processing

showInteract = True          # default for interactive use
showBatch = False            # default for batch use

showTrace = None             # control flag

# Trace Output

def TOshow(mesg, text):
    '''write trace message to output if enabled'''
    if showTrace:
        UIshow( f'{mesg:15s}', text )
        
def TOstring(tag, this):
    
    if showTrace:
        TOshow( tag, ' '.join([str(e) for e in this]) )

# -----------------------
# Parse Tracing
# -----------------------

def PTshowexpr(this):

    TOshow( 'Parse', visSep )
    TOshow( 'Current Expr', this )

def PTshowparse(ok, res, stk):

    if ok:
        TOstring( 'Current RPN', res )
        TOstring( 'Operator Stack', stk )

def PTshowtoken(this):

    if not this[0] == ' ':
        TOshow( "Found Token", this )

# -----------------------
# Evaluation Tracing
# -----------------------

def ETshowtoken(this):
    
    TOshow( 'Eval', visSep )
    TOshow( 'Current token', this )

def ETshoweval(stk):
    
    TOstring( 'Operand Stack', stk )


# Parser

In [ ]:
# operands accepted:
# - decimal and hexadecimal integer literals
# - decimal and hexadecimal floating point literals

# operators accepted:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - grouping parentheses
# - logical not, equality, inequality

# errors detected:
# - unrecognized input
# - out of range numeric input
# - malformed expression

# result tuple:
# - (True, [parse])
# - (False, None)

class Parser(object):
    
    VERSIONNUMBER = '8.00'
    
    _FLTMAX =  4294967295                # 2**32-1
    _FLTMIN = -4294967296                # -(2**32)

    _unPrefxOp = '[-+!]'                                   # unary operators
 
    _unPrec = { '-': 80, '+': 80, '!': 80 }                # unary operator precedence
    
    _binInfxOp = '[-+*/]|==|!='                            # binary operators
        
    _binPrec = {
        '*': 70, '/': 70,
        '-': 60, '+': 60,
        '==': 50, '!=': 50
    }

    def __init__(self):
        pass
           
    def doparse(self, this):

        # initialize

        expr = this                # save to new variable but retain original for error reports
        start = 15                 # tracked so we can report where in an expression an error occurred
        token = None               # anything successfully matched
        ok = wantoperand = True    # flags
        result = []                # rpn expression
        stk = [ ('EOE', 1) ]       # operator stack

        def parseErr(mesg):
            '''report parse error'''
            UIerror(mesg)
            UIwriteln(f'>>> {this}')
            UIwriteln(f'{"^^near here".rjust(start)}')
            return False

        def popGEop(prec):
            '''pop operators of equal or greater precedence'''
            while prec <= stk[-1][1]:
                result.append(stk.pop()[0])

        def pushLeft(op, prec):
            '''push left associative operator on stack'''
            popGEop(prec)
            stk.append( (op, prec) )

        def popGop(prec):
            '''pop operators of greater precedence'''
            while prec < stk[-1][1]:
                result.append(stk.pop()[0])

        def pushRight(op, prec):
            '''push right associative operator on stack'''
            popGop(prec)
            stk.append( (op, prec) )

        def popUntil(op, prec):
            '''clear and check operator stack'''
            popGEop(prec)
            if op == stk.pop()[0]:      # top remaining operator is the one we want to see ?
                return True
            elif op == '(':
                return parseErr('Unmatched right parenthesis')
            elif op == 'EOE':
                return parseErr('Unmatched left parenthesis')
       
        def convertFloat(ulit, base):
            '''convert numeric literal to internal form'''
                    
            # separate integer and fractional portions
            
            lint, _, lfrc = ulit.upper().partition('.')
            
            # convert integer portion (if any)
 
            uint = 0
            if lint != '':
                p = re.search('[1-9A-F][0-9A-F]*', lint )
                if p != None:
                    for ch in p.group():
                        digval = '0123456789ABCDEF'.find(ch)
                        if uint <= (self._FLTMAX - digval)/base:
                            uint =  uint * base + digval
                        else:
                            return parseErr(f'\'{ulit}\' is out of range')
                    
            # convert fractional portion (if any)
                    
            ufrc = 0
            if lfrc != '':
                fbase = 1
                for ch in lfrc:
                    digval = '0123456789ABCDEF'.find(ch)
                    fbase *= base
                    ufrc += digval/fbase
        
            if uint == self._FLTMAX and ufrc != 0:
                return parseErr(f'\'{ulit}\' is out of range')
                       
            result.append(uint + ufrc)
            return True

        def startsWith(regex):
            '''test if expression starts with given regular expression'''
            nonlocal expr, start, token

            p = re.match(regex, expr)
            if p == None:
                return False
            else:
                token = p.group()              # what we matched
                start += len(token)            # update to next match position in original string
                expr = expr[len(token):]       # "chop off" what we matched
                PTshowtoken(token)             # trace
                return True

        # top level main loop

        while ok and len(expr):

            _ = startsWith('[ ]+')                             # skip leading whitespace

            PTshowexpr(expr)                                   # trace

            # look for operand

            if wantoperand:

                if startsWith('[(]'):
                    '''left parenthesis ?'''
                    stk.append( ('(', 2) )


                elif startsWith(self._unPrefxOp):
                    '''unary prefix ?'''
                    pushRight( 'U' + token, self._unPrec[token] )

                else:

                    wantoperand = False                         # flip
                    
                    if startsWith('0[xX]([0-9a-fA-F]+([.][0-9a-fA-F]*)?|[.][0-9a-fA-F]+)'):
                        '''unsigned hexadecimal literal ?'''
                        ok = convertFloat(token, 16)

                    elif startsWith('[0-9]+([.][0-9]*)?|[.][0-9]+'):
                        '''unsigned decimal literal?'''
                        ok = convertFloat(token, 10)

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operand')

            # look for operator

            else:

                if startsWith('[)]'):
                    ok = popUntil( '(', 4 )

                else:

                    wantoperand = True                                # flip

                    if startsWith(self._binInfxOp):
                        '''binary infix ?'''
                        pushLeft( 'B' + token, self._binPrec[token] )

                    else:
                        '''malformed'''
                        ok = parseErr('Expecting operator')

            PTshowparse(ok, result, stk )                         # trace

        if ok:
            if wantoperand:
                ok = parseErr('Unexpected end of expression')     # must be in 'wantoperator' state   
            else:
                ok = popUntil( 'EOE', 3 )                         # clear operator stack

        return (ok, result if ok else None)                       # done

### How it works

The first thing we need to decide on is what floating point literals look like. In this version they will look like integer literals, except they may optionally have a radix point which itself may be optionally followed by more digits.

For decimal floating point, the regular expression we use to recognize them is:

```Python
'[0-9]+([.][0-9]*)?|[.][0-9]+'
```

which means "one or more digits, optionally followed by a decimal point, which can optionally be followed by zero or more digits, *OR* a decimal point followed by one or more digits". This recognizes all of these forms as decimal floating point numbers:

- 123
- 123.
- 123.4
- .456

The regular expression for recognizing hexadecimal floating point numbers is essentially the same, just extended to the 16 hexadecimal digits.

Note that our original integer literals are also recognized by these expressions. If we adapt our integer conversion function (now renamed from *convertUint()* to *convertFloat()*) to handle floating points as well, we can keep using just two checks for numeric literals in the main loop instead of adding a third just for floating point. So that's what we do.

In *convertFloat()* we use the string method *partition()* to separate the integer and fractional portions of the input literal at the radix point (we also take the opportunity to uppercase any hexadecimal digits).

>We prefer *partition()* to *split()* because the former has well-defined behavior if the input literal doesn't actually contain a radix point. What *split()* does in that case is not well-documented.

If there is no radix point, the entire input literal will be in *lint* and *lfrc* will be null. The same will be true if there are no digits following the radix point. If there are, then both *lint* and *lfrc* will be non-null. Finally, if the input literal begins with a radix point, *lint* will be null and *lfrc* will not be.

>In no case do we actually care about the radix point itself, so we conventionally assign it to the *_* variable.

Conversion of any integer portion of the input literal proceeds exactly as it did in previous versions. Conversion of the fractional portion is similar, but here we divide each successive digit by a larger and larger value to reflect its decreasing significance.

>We don't need to check for overflow during this step because the value of the fractional part will always be between zero and one. Underflow, on the other hand, is quite possible. As the divisor grows, eventually each successive digit will have a value after the division indistinguishable from zero because of the way floating point arithmetic works.

The final value after conversion is simply the sum of the integer and fractional portions. We make one final check. If the value of the integer portion is anything other than *_FLTMAX*, no value of the fractional portion can make the sum of the two exceed *_FLTMAX*. However if it is *_FLTMAX*, any non-zero value of the fractional part will cause overflow.

>Of course a major advantage of a floating point representation is that it increases the range of represented numbers beyond that of simple integers. So why limit them to the same range here? Mostly because what we're really interested in here is simply how to recognize and convert them. Also because leaving them as they are still serves our range checking purposes. We do rename the limits from *_INT---* to *_FLT---*, though.

# Evaluator

In [ ]:
# operators handled:
# - unary negation, plus
# - binary addition, subtraction, multiplication, division
# - logical negation, equality and inequality

# errors detected:
# - out of range
# - division by zero

# return tuple:
# - (True, result)
# - (False, None)

class Evaluator(Parser):
    
    def __init__(self):
        pass
 
    def doeval(self, rpn):
 
        def inRange(ok, val):
            '''range check test result'''
            if ok:
                stk.append( val )
            else:
                UIerror( 'Evaluation result out of range' )
            return ok
        
        def pushOperand(val):
            '''push operand on stack'''
            stk.append( val )
            return True

#        def unPlu(arg):
#            '''unary plus'''
#           return pushOperand( arg )

        def unNeg(arg):
            '''unary negation'''
            return inRange( arg != self._FLTMIN, -arg )
        
        def unNot(arg):
            '''logical not'''
            return pushOperand( not arg )

        def binAdd(rgt, lft):
            '''binary addition'''
            if lft >= 0:
                return inRange( rgt <= self._FLTMAX - lft, lft+rgt )       
            else:
                return inRange( rgt >= self._FLTMIN - lft, lft+rgt )

        def binSub(rgt, lft):
            '''binary subtraction'''
            if lft >= 0:
                return inRange( lft - self._FLTMAX <= rgt, lft-rgt )
            elif rgt >= 0:
                return inRange( lft - self._FLTMIN >= rgt, lft-rgt )
            
            # negative minus negative
            # required: lft - rgt <= FLTMAX
            
            else:
                return inRange( lft <= self._FLTMAX + rgt, lft-rgt )

        def binMul(rgt, lft):
            '''binary multiplication'''
            if abs(lft) <= 1 or abs(rgt) <= 1:
                return pushOperand( lft * rgt )

            if lft > 0:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMAX / lft, lft*rgt )
                else:
                    return inRange( rgt >= self._FLTMIN / lft, lft*rgt )

            else:
                if rgt > 0:
                    return inRange( rgt <= self._FLTMIN / lft, lft * rgt )
                else:
                    return inRange( rgt >= self._FLTMAX / lft, lft * rgt )

        def binDiv(rgt, lft):
            '''binary division'''
            if abs(rgt) >= 1:
                return pushOperand( lft/rgt )
            
            # rgt positive ?
            
            elif rgt > 0:
                
                # lft positive ?
                # req: lft/rgt <= FLTMAX
                
                if lft > 0 :
                    return inRange( lft <= self._FLTMAX * rgt, lft / rgt )
                
                # req: lft/rgt >= FLTMIN
                
                else:
                    return inRange( lft >= self._FLTMIN * rgt, lft / rgt )
                
            # rgt negative ?
            
            elif rgt < 0:
                
                # lft positive ?
                # req: lft/rgt >= FLTMIN
                
                if lft > 0:
                    return inRange( lft <= self._FLTMIN * rgt, lft / rgt )       # reverse inequality
                 
                # req: lft/rgt <= FLTMAX
                
                else:
                    return inRange( lft >= self._FLTMAX * rgt, lft / rgt )        # reverse inequality
                
            # rgt is zero

            else:
                UIerror( 'Division by zero' )
                return False
            
        def binEqu(rgt, lft):
            '''logical equality'''
            return pushOperand( lft == rgt )
        
        def binNeq(rgt, lft):
            '''logical inequality'''
            return pushOperand( lft != rgt )
        
        # initialize
                                
        unDispatch = {
            'U-': unNeg,
            'U+': pushOperand,
            'U!': unNot
        }
        
        binDispatch = {
            'B+': binAdd,
            'B-': binSub,
            'B*': binMul,
            'B/': binDiv,
            'B==': binEqu,
            'B!=': binNeq
        }
  
        stk = []
        ok = True

        # main loop
        
        for v in rpn:

            ETshowtoken(v)
            
            if v in binDispatch:
                ok = binDispatch[v](stk.pop(), stk.pop())
                
            elif v in unDispatch:
                ok = unDispatch[v](stk.pop())
                
            else:
                stk.append( v )

            if not ok:
                return ( False, None )

            ETshoweval( stk )

        return ( True, stk.pop() )


### How it works

The evaluator has no new capabilities, but there are new implications for subtraction, multiplication and division.

For subtraction, overflow is now possible in some cases of a negative minus a negative. Because the absolute value of *INTMIN* is greater by one than *INTMAX*, subtracting *INTMIN* from any value between minus one and zero will overflow:

```Python
x - INTMIN >= INTMAX for -1 < x < 0
```

To handle this possibility we add a third branch to *binSub()* specifically for negative minus negative cases.

For multiplication, overflow is not possible if either operand has an absolute value of less than one, since the absolute value of the result is necessarily less than the absolute value of the other operand. Possibly we don't need to make any changes at all. But we can also replace the required test we already have for a left operand of zero with a very similar one which covers every operand value between minus one and one. Which seems simple and worthwhile doing.

For division, the possibility is reversed. Overflow might occur if the divisor has an absolute value less than one. We now have to guard against that in a way we did not for integer division. The tests are straightforward and quite similar in form to the multiplication checks, including the reversal of the inequality direction when *rgt* is negative. We also now allow the return value to be non-integer.

## Running the parser

In [ ]:
passCnt = failCnt = 0                       # most useful for test input files, but never any harm

myParser = myEvaluator = None               # where we keep instances of our classes

def startUp(flag):
    '''begin execution'''
    global passCnt, failCnt, showTrace
    global myParser, myEvaluator
    if not myParser:
        myParser = Parser()
    if not myEvaluator:
        myEvaluator = Evaluator()
    UIshow( 'Parser', myParser.VERSIONNUMBER )
    passCnt = failCnt = 0
    showTrace = flag
    
def shutDown():
    '''terminate execution'''
    UIwriteSep()
    UIshow( 'Pass', passCnt )
    UIshow( 'Fail', failCnt )
    
# run parser

def parseOne(this):
    '''parse/evaluate one expression'''
    global passCnt, failCnt
    UIwriteSep()
    UIshow( 'Input', this )
    ok, res = myParser.doparse( this )
    if ok:
        UIshow( 'Final Parse', res )
        ok, res = myEvaluator.doeval( res )
        if ok:
            UIshow( 'Final Eval', res )
    if ok:
        passCnt += 1
    else:
        failCnt += 1

## Interactive use

In [ ]:
def parse():
    
    startUp(showInteract)
    while True:
        inp = input( 'Expression: ' )
        UIwriteln( '' )                      # looks better with a blank line here
        if inp.upper()[0] == 'Q':
            break
        elif inp.strip():
            parseOne( inp )
    shutDown()

## Batch processing

In [ ]:
testDir = '..\\ParserTest\\'            # directory holding test input files (empty string if same as notebook directory)

# convert current version number to match test file numbers
# - done this way so we can update only the version number and everything still works

def currNum():
    
    head = myParser.VERSIONNUMBER[:len(myParser.VERSIONNUMBER)-3]
    tail = myParser.VERSIONNUMBER[-2:]
    return f'{head:0>2}{tail}'

# make full path name to test file

def makePath(typ, num):
    return f'{testDir}{typ}{num}.txt'

# run one test

def runTest(this):
    
    UIwriteln(f'Parser {myParser.VERSIONNUMBER} vs {this[-12:-4]}')
    
    with open(this) as f:
        data = f.readlines()
    for line in data:
        test = line.strip()
        if test and test[0] != '#':         # skip blank and comment lines
            parseOne(test)
    
# run a test of current or specified version which should succeed
    
def good(num='curr'):
  
    startUp(showBatch)
    runTest(makePath('pass', currNum() if num == 'curr' else num))
    shutDown()
    
# run a test of current or specified version which should fail

def bad(num='curr'):
    
    startUp(showBatch)
    runTest(makePath('fail', currNum() if num == 'curr' else num))
    shutDown()
    
# run regression test against current and all previous test files

def regress():
            
    UIwriteln('PASS tests')
    
    startUp(showBatch)                       # must create objects before we can access variables inside them 
    currFn = makePath('pass', currNum())
    failed = []
    fnlist = glob.glob(f'{testDir}pass????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = failCnt
            runTest(fn)
            if atstart < failCnt:
                failed.append(fn)               
    shutDown()
    
    UIwriteln('FAIL tests')
    
    startUp(showBatch)
    currFn = makePath('fail',currNum())
    passed = []
    fnlist = glob.glob(f'{testDir}fail????.txt')
    for fn in fnlist:
        if fn <= currFn:
            atstart = passCnt
            runTest(fn)
            if atstart < passCnt:
                passed.append(fn)                
    shutDown()
    
    if not len(failed):
        UIwriteln('All pass tests succeded')
    else:
        UIwriteln('Pass tests which failed')
        for fn in failed:
            UIwriteln(f'  {fn}')
            
    if not len(passed):
        UIwriteln('All fail tests succeded')
    else:
        UIwriteln('Fail tests which passed')
        for fn in passed:
            UIwriteln(f'   {fn}')
              

# Testing the parser

In [ ]:
parse()       # interactive, one expression at a time

In [ ]:
good()        # run current parser against its own pass test. Use good('1234') to run against specific pass test.

In [ ]:
bad()         # run current parser against its own fail test. Use bad('5678') to run against specific fail test.

In [ ]:
regress()     # run parser against all previous tests